In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import sklearn.metrics as m
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
import pickle

In [2]:
df = pd.read_csv('df_kapremont_manyfloat_ver_5.csv')
df

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,414.0,0.316406,3
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,414.0,0.316406,5
2,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подъездов, направленный на восстановлен...",12,414.0,0.316406,5
3,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем водоотв...,12,414.0,0.316406,3
4,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"замена оконных блоков, расположенных в помещен...",12,414.0,0.316406,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43575,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт крыши,12,126.0,0.067448,11
43576,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,"ремонт подвальных помещений, относящихся к общ...",12,126.0,0.067448,5
43577,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт фасадов,12,126.0,0.067448,1
43578,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт внутридомовых инженерных систем теплосн...,12,126.0,0.067448,4


In [3]:
df.UNOM.value_counts()

24239    2280
2930      994
64039     957
24750     910
15109     801
         ... 
18528       4
27141       2
24135       2
28691       2
21908       1
Name: UNOM, Length: 371, dtype: int64

In [4]:
len(df.WORK_NAME.value_counts())

19

In [5]:
df.head(2)

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,414.0,0.316406,3
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,414.0,0.316406,5


In [6]:
df['Возраст дома'] = 2022 - df['Год постройки']
df

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,414.0,0.316406,3,49.0
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,414.0,0.316406,5,49.0
2,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"ремонт подъездов, направленный на восстановлен...",12,414.0,0.316406,5,49.0
3,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем водоотв...,12,414.0,0.316406,3,49.0
4,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,"замена оконных блоков, расположенных в помещен...",12,414.0,0.316406,1,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43575,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт крыши,12,126.0,0.067448,11,47.0
43576,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,"ремонт подвальных помещений, относящихся к общ...",12,126.0,0.067448,5,47.0
43577,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт фасадов,12,126.0,0.067448,1,47.0
43578,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,28393,в эксплуатации,0.0,2022-12-20,ремонт внутридомовых инженерных систем теплосн...,12,126.0,0.067448,4,47.0


In [7]:
#кодирование таргета
encoder = LabelEncoder()
df['WORK_NAME_TARGET'] = encoder.fit_transform(df.WORK_NAME)
df

,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,...,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,WORK_NAME,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
0,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем электро...,12,414.0,0.316406,3,49.0,12
1,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"ремонт подвальных помещений, относящихся к общ...",12,414.0,0.316406,5,49.0,15
2,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"ремонт подъездов, направленный на восстановлен...",12,414.0,0.316406,5,49.0,16
3,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,ремонт внутридомовых инженерных систем водоотв...,12,414.0,0.316406,3,49.0,3
4,1973.0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,...,в эксплуатации,0.0,2022-12-02,"замена оконных блоков, расположенных в помещен...",12,414.0,0.316406,1,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43575,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт крыши,12,126.0,0.067448,11,47.0,13
43576,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,"ремонт подвальных помещений, относящихся к общ...",12,126.0,0.067448,5,47.0,15
43577,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт фасадов,12,126.0,0.067448,1,47.0,18
43578,1975.0,И-209А,14.0,1.0,4263.0,4263.0,0.0,каменные и бетонные,2.0,0.0,...,в эксплуатации,0.0,2022-12-20,ремонт внутридомовых инженерных систем теплосн...,12,126.0,0.067448,4,47.0,8


In [8]:
work_name = df[['WORK_NAME', 'WORK_NAME_TARGET']].groupby('WORK_NAME_TARGET')['WORK_NAME'].agg(lambda x: x)
work_name_key = dict()

In [9]:
work_name.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype='int64', name='WORK_NAME_TARGET')

In [10]:
df = df.drop(['Год постройки', 'WORK_NAME'], axis=1)
df.head()

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
0,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,414.0,0.316406,3,49.0,12
1,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,414.0,0.316406,5,49.0,15
2,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,414.0,0.316406,5,49.0,16
3,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,414.0,0.316406,3,49.0,3
4,I-515,9.0,12.0,21103.0,21088.0,15.0,панельные,12.0,0.0,22830,в эксплуатации,0.0,2022-12-02,12,414.0,0.316406,1,49.0,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43580 entries, 0 to 43579
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            43580 non-null  object 
 1   Количество этажей                        43580 non-null  float64
 2   Количество подъездов                     43580 non-null  float64
 3   Общая площадь                            43580 non-null  float64
 4   Общая площадь жилых помещений            43580 non-null  float64
 5   Общая площадь нежилых помещений          43580 non-null  float64
 6   Материал стен                            43580 non-null  object 
 7   Количество пассажирских лифтов           43580 non-null  float64
 8   Количество грузопассажирских лифтов      43580 non-null  float64
 9   UNOM                                     43580 non-null  int64  
 10  Статус МКД                               43580

In [12]:
#замена данных на инт
for col in ['Количество этажей', 'Количество подъездов', 'Количество пассажирских лифтов',
         'Количество грузопассажирских лифтов', 'Количество грузовых лифтов']:
    df.loc[df[col] == 'нет данных', col] = '-1'
    df[col] = df[col].astype('float32').astype('int32')

In [13]:
#замена данных на флоат
for col in ['Общая площадь', 'Общая площадь жилых помещений', 'Общая площадь нежилых помещений']:
    df.loc[df[col] == 'нет данных', col] = '-1'
    df[col] = df[col].astype('float32').astype('int32')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43580 entries, 0 to 43579
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            43580 non-null  object 
 1   Количество этажей                        43580 non-null  int32  
 2   Количество подъездов                     43580 non-null  int32  
 3   Общая площадь                            43580 non-null  int32  
 4   Общая площадь жилых помещений            43580 non-null  int32  
 5   Общая площадь нежилых помещений          43580 non-null  int32  
 6   Материал стен                            43580 non-null  object 
 7   Количество пассажирских лифтов           43580 non-null  int32  
 8   Количество грузопассажирских лифтов      43580 non-null  int32  
 9   UNOM                                     43580 non-null  int64  
 10  Статус МКД                               43580

In [15]:
df.loc[df['Серия проекта'] == 'нет данных', 'Серия проекта'] = 'нет серии проекта'
df.loc[df['Материал стен'] == 'нет данных', 'Материал стен'] = 'нет информации по материалам стен'

In [16]:
df

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Дата создания во внешней системе,Месяц создания во внешней системе,Количество обращений за последний месяц,Среднемесячное количество обращений,PLAN_MONTH_START,Возраст дома,WORK_NAME_TARGET
0,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,2022-12-02,12,414.0,0.316406,3,49.0,12
1,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,2022-12-02,12,414.0,0.316406,5,49.0,15
2,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,2022-12-02,12,414.0,0.316406,5,49.0,16
3,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,2022-12-02,12,414.0,0.316406,3,49.0,3
4,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,2022-12-02,12,414.0,0.316406,1,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43575,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,2022-12-20,12,126.0,0.067448,11,47.0,13
43576,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,2022-12-20,12,126.0,0.067448,5,47.0,15
43577,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,2022-12-20,12,126.0,0.067448,1,47.0,18
43578,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,2022-12-20,12,126.0,0.067448,4,47.0,8


In [17]:
df = df.drop(['Дата создания во внешней системе', 'Месяц создания во внешней системе', 'PLAN_MONTH_START'], axis=1)
df

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома,WORK_NAME_TARGET
0,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,414.0,0.316406,49.0,12
1,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,414.0,0.316406,49.0,15
2,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,414.0,0.316406,49.0,16
3,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,414.0,0.316406,49.0,3
4,I-515,9,12,21103,21088,15,панельные,12,0,22830,в эксплуатации,0,414.0,0.316406,49.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43575,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,126.0,0.067448,47.0,13
43576,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,126.0,0.067448,47.0,15
43577,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,126.0,0.067448,47.0,18
43578,И-209А,14,1,4263,4263,0,каменные и бетонные,2,0,28393,в эксплуатации,0,126.0,0.067448,47.0,8


In [18]:
#сохранение обучающего датафрейма
df.to_csv('df_train_pred_work.csv', index=False)

In [19]:
df.columns

Index(['Серия проекта', 'Количество этажей', 'Количество подъездов',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Общая площадь нежилых помещений', 'Материал стен',
       'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов',
       'UNOM', 'Статус МКД', 'Количество грузовых лифтов',
       'Количество обращений за последний месяц',
       'Среднемесячное количество обращений', 'Возраст дома',
       'WORK_NAME_TARGET'],
      dtype='object')

## Обучение модели

In [20]:
x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['WORK_NAME_TARGET'], axis = 1), df['WORK_NAME_TARGET'], test_size = 0.30, random_state = 13)
x_test

,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,UNOM,Статус МКД,Количество грузовых лифтов,Количество обращений за последний месяц,Среднемесячное количество обращений,Возраст дома
38409,II-18-01/12,12,1,3607,3607,0,из железобетонных сегментов,2,0,26960,в эксплуатации,0,152.0,0.108333,53.0
39888,индивидуальный проект,16,9,22761,22484,277,кирпичные,9,7,64039,в эксплуатации,0,957.0,0.730469,25.0
16486,II-49-06,9,12,20917,20754,163,панельные,12,0,24761,в эксплуатации,0,215.0,0.335938,51.0
18114,П-43,16,1,5869,5836,33,панельные,1,1,17488,в эксплуатации,0,256.0,0.316667,41.0
31711,П-49 Д,9,6,10358,10358,0,панельные,6,0,382,в эксплуатации,0,232.0,0.272917,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15738,нет серии проекта,5,3,2599,2484,115,кирпичные,0,0,19274,в эксплуатации,0,18.0,0.014063,60.0
31212,индивидуальный проект,4,2,2997,1817,1180,кирпичные,0,0,6329,в эксплуатации,0,406.0,0.138542,65.0
38691,И-209А,14,1,4269,4269,0,из железобетонных сегментов,2,0,28406,в эксплуатации,0,143.0,0.103125,46.0
40675,нет серии проекта,16,1,6717,6021,696,панельные,1,1,28562,в эксплуатации,0,120.0,0.101042,55.0


In [21]:
df.isna().sum()

Серия проекта                              0
Количество этажей                          0
Количество подъездов                       0
Общая площадь                              0
Общая площадь жилых помещений              0
Общая площадь нежилых помещений            0
Материал стен                              0
Количество пассажирских лифтов             0
Количество грузопассажирских лифтов        0
UNOM                                       0
Статус МКД                                 0
Количество грузовых лифтов                 0
Количество обращений за последний месяц    0
Среднемесячное количество обращений        0
Возраст дома                               0
WORK_NAME_TARGET                           0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43580 entries, 0 to 43579
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Серия проекта                            43580 non-null  object 
 1   Количество этажей                        43580 non-null  int32  
 2   Количество подъездов                     43580 non-null  int32  
 3   Общая площадь                            43580 non-null  int32  
 4   Общая площадь жилых помещений            43580 non-null  int32  
 5   Общая площадь нежилых помещений          43580 non-null  int32  
 6   Материал стен                            43580 non-null  object 
 7   Количество пассажирских лифтов           43580 non-null  int32  
 8   Количество грузопассажирских лифтов      43580 non-null  int32  
 9   UNOM                                     43580 non-null  int64  
 10  Статус МКД                               43580

In [23]:
len(y_train.unique())

19

In [24]:
#сохранение тестового датафрейма
x_test.to_csv('df_test_pred_work.csv', index=False)

In [25]:
cat_features = ['Серия проекта', 'Количество этажей', 'Количество подъездов',
                'Материал стен', 'UNOM', 'Статус МКД']

In [26]:
train_pool = Pool(x_train, y_train, cat_features=cat_features)

In [59]:
%%time
model = CatBoostClassifier(loss_function='MultiClass', random_seed=13)
model.fit(train_pool, verbose = 100)
pred = model.predict(x_test)

Learning rate set to 0.0777
0:	learn: 2.8916147	total: 142ms	remaining: 2m 21s
100:	learn: 1.7442829	total: 7.56s	remaining: 1m 7s
200:	learn: 1.2214097	total: 19.1s	remaining: 1m 15s
300:	learn: 0.8700561	total: 32.7s	remaining: 1m 15s
400:	learn: 0.6553662	total: 45.8s	remaining: 1m 8s
500:	learn: 0.5133516	total: 1m	remaining: 1m
600:	learn: 0.4114545	total: 1m 10s	remaining: 46.6s
700:	learn: 0.3358915	total: 1m 22s	remaining: 35.2s
800:	learn: 0.2821189	total: 1m 35s	remaining: 23.7s
900:	learn: 0.2406145	total: 1m 45s	remaining: 11.6s
999:	learn: 0.2087894	total: 1m 55s	remaining: 0us
CPU times: user 6min 26s, sys: 2.31 s, total: 6min 29s
Wall time: 1min 55s


In [60]:
print('F1 score -', f1_score(y_test, pred, average=None))

F1 score - [0.78571429 0.20689655 0.         0.05882353 0.         0.
 0.08       0.16666667 0.12820513 0.         0.04878049 0.
 0.         0.19607843 0.18181818 0.20588235 0.         0.
 0.11538462]


In [61]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.73      0.85      0.79        39
           1       0.18      0.25      0.21        12
           2       0.00      0.00      0.00         3
           3       0.08      0.05      0.06        22
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         2
           6       0.10      0.07      0.08        29
           7       0.33      0.11      0.17         9
           8       0.13      0.12      0.13        40
           9       0.00      0.00      0.00         2
          10       0.03      0.09      0.05        22
          11       0.00      0.00      0.00         7
          12       0.00      0.00      0.00         8
          13       0.17      0.24      0.20        21
          14       1.00      0.10      0.18        10
          15       0.19      0.23      0.21        31
          16       0.00      0.00      0.00         5
          17       0.00    

/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Список работ с вероятностями

In [62]:
pred_proba = model.predict_proba(x_test)
pred_proba

array([[2.34342636e-02, 3.02787559e-02, 2.39770924e-03, ...,
        1.53473542e-02, 1.06455158e-03, 3.05013040e-01],
       [1.71501551e-02, 9.25838100e-03, 2.79935334e-03, ...,
        7.49687328e-03, 6.02604897e-03, 1.22555976e-01],
       [9.96197437e-01, 3.77756287e-04, 5.14183617e-05, ...,
        7.09469471e-05, 7.11584798e-05, 6.96888119e-04],
       ...,
       [3.25751216e-02, 4.36257408e-02, 5.43283758e-03, ...,
        7.89190512e-03, 1.39544380e-03, 2.27400046e-01],
       [1.11655759e-03, 5.97349998e-03, 1.01797934e-02, ...,
        6.83372178e-03, 3.33964077e-03, 3.79958029e-02],
       [4.58077875e-03, 3.56228594e-02, 4.79794298e-03, ...,
        4.62778639e-02, 2.91931674e-03, 5.68303190e-02]])

In [63]:
len(pred_proba)

300

In [64]:
len(pred_proba[0])

19

In [65]:
#получение json словаря
with open('work_name_key.json') as f:
    templates = json.load(f)

print(templates)

{'0': 'замена лифтового оборудования', '1': 'замена оконных блоков, расположенных в помещениях общего пользования', '2': 'ремонт внутреннего водостока', '3': 'ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)', '4': 'ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки)', '5': 'ремонт внутридомовых инженерных систем газоснабжения', '6': 'ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали)', '7': 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)', '8': 'ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)', '9': 'ремонт внутридомовых инженерных систем теплоснабжения (стояки)', '10': 'ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)', '11': 'ремонт внутридомовых инженерных систем холодного водоснабжения (стояки)', '12': 'ремонт внутридомовых инженерных систем электроснабжения', '13': 'ремонт крыши', 

In [76]:
def creat_predict_proba(pred_prob, ind_pred, x_test, key_dict):
    df_final = pd.DataFrame(data=pred_prob[ind_pred], columns=['Вероятность'])
    df_final['Наименование работ'] = key_dict
    df_final['Наименование работ'] = df_final['Наименование работ'].map(key_dict)
    df_final['UNOM'] = x_test.iloc[ind_pred].UNOM
    df_final = df_final.sort_values('Вероятность', ascending=False)
    return df_final
    

In [77]:
x_test.iloc[1]

Серия проекта                                                   И-700А
Количество этажей                                                   19
Количество подъездов                                                 2
Общая площадь                                                     8688
Общая площадь жилых помещений                                     8638
Общая площадь нежилых помещений                                     50
Материал стен                              из железобетонных сегментов
Количество пассажирских лифтов                                       2
Количество грузопассажирских лифтов                                  2
UNOM                                                              8416
Статус МКД                                              в эксплуатации
Количество грузовых лифтов                                           0
Количество обращений за последний месяц                          180.0
Среднемесячное количество обращений                            0.16875
Возрас

In [78]:
creat_predict_proba(pred_proba, 0, x_test, templates)

,Вероятность,Наименование работ,UNOM
18,0.305013,ремонт фасадов,30007
13,0.147918,ремонт крыши,30007
15,0.111981,"ремонт подвальных помещений, относящихся к общ...",30007
6,0.106651,ремонт внутридомовых инженерных систем горячег...,30007
10,0.099014,ремонт внутридомовых инженерных систем холодно...,30007
8,0.042019,ремонт внутридомовых инженерных систем теплосн...,30007
12,0.038901,ремонт внутридомовых инженерных систем электро...,30007
3,0.035265,ремонт внутридомовых инженерных систем водоотв...,30007
1,0.030279,"замена оконных блоков, расположенных в помещен...",30007
0,0.023434,замена лифтового оборудования,30007


In [ ]:
model

In [24]:
#сохраняем моделb lgbm
with open('jkh_catboost.pkl', 'wb') as file:
    pickle.dump(model, file)